In [1]:
#https://www.kaggle.com/gbonesso/enem-2016

In [4]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

from pyspark import SparkContext, SQLContext, SparkConf, StorageLevel
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import glob, os

In [5]:
conf = SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '4'), 
                                   ('spark.cores.max', '4'), ('spark.driver.memory','8g')])
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
os.chdir("./Datasets")
sc.getConf().getAll()

FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: './Datasets'

In [ ]:
rdd = sc.textFile("./Datasets/microdados_enem_2016_coma.csv", 100).persist()

In [10]:
rdd.take(1)

['NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,IN_OUTRA_DEF,IN_SABATISTA,IN_GESTANTE,IN_LACTANTE,IN_IDOSO,IN_ESTUDA_CLASSE_HOSPITALAR,IN_SEM_RECURSO,IN_BRAILLE,IN_AMPLIADA_24,IN_AMPLIADA_18,IN_LEDOR,IN_ACESSO,IN_TRANSCRICAO,IN_LIBRAS,IN_LEITURA_LABIAL,IN_MESA_CADEIRA_RODAS,IN_MESA_CADEIRA_SEPARADA,IN_APOIO_PERNA,IN_GUIA_INTERPRETE,IN_MACA,IN_COMPUTADOR,IN_CADEIRA_ESPECIAL,IN_CADEIRA_CANHOTO,IN_CADEIRA_ACOLCHOA

### Map dos dados relevantes:

In [11]:
rddEnemMap = rdd.map(lambda line: (line.split(",")[1],line.split(",")[3],
                                   line.split(",")[7],line.split(",")[8],line.split(",")[9],line.split(",")[10],
                         line.split(",")[-52],line.split(",")[-53],line.split(",")[-54],line.split(",")[-55]))

In [12]:
rddEnemMap.take(5)

[('NU_ANO',
  'NO_MUNICIPIO_RESIDENCIA',
  'TP_SEXO',
  'TP_ESTADO_CIVIL',
  'TP_COR_RACA',
  'TP_NACIONALIDADE',
  'NU_NOTA_COMP5',
  'NU_NOTA_COMP4',
  'NU_NOTA_COMP3',
  'NU_NOTA_COMP2'),
 ('2016', 'Jo�o Pessoa', 'M', '0', '3', '1', '120', '120', '100', '120'),
 ('2016', 'Vitorino', 'M', '0', '1', '1', '80', '120', '120', '120'),
 ('2016', 'Salvador', 'M', '0', '3', '1', '100', '80', '80', '100'),
 ('2016', 'Bel�m', 'M', '0', '1', '1', '120', '140', '120', '120')]

### Validando quantidade de partições

In [13]:
numPartitions = rddEnemMap.getNumPartitions()
print("Total partitions: {}".format(numPartitions))

Total partitions: 170


### Utilizando coalesce para evitar o shuffle dos dados:

In [15]:
rddEnemMap_coalesce = rddEnemMap.coalesce(55)

In [16]:
numPartitions = rddEnemMap_coalesce.getNumPartitions()
print("Total partitions: {}".format(numPartitions))

Total partitions: 55


In [17]:
df = spark.createDataFrame(rddEnemMap_coalesce)

In [18]:
df.show()

+------+--------------------+-------+---------------+-----------+----------------+-------------+-------------+-------------+-------------+
|    _1|                  _2|     _3|             _4|         _5|              _6|           _7|           _8|           _9|          _10|
+------+--------------------+-------+---------------+-----------+----------------+-------------+-------------+-------------+-------------+
|NU_ANO|NO_MUNICIPIO_RESI...|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|NU_NOTA_COMP5|NU_NOTA_COMP4|NU_NOTA_COMP3|NU_NOTA_COMP2|
|  2016|         Jo�o Pessoa|      M|              0|          3|               1|          120|          120|          100|          120|
|  2016|            Vitorino|      M|              0|          1|               1|           80|          120|          120|          120|
|  2016|            Salvador|      M|              0|          3|               1|          100|           80|           80|          100|
|  2016|               Bel�

### Salvando Rdd particionado e com dados relevantes:

In [19]:
df.write.csv("./Datasets/enem_importcoluns_load", sep=',', header=True, mode="overwrite")

In [20]:
rdd = sc.textFile("./Datasets/enem_importcoluns_load")

In [22]:
numPartitions = rdd.getNumPartitions()
print("Total partitions: {}".format(numPartitions))

Total partitions: 55


In [23]:
rdd.take(5)

['_1,_2,_3,_4,_5,_6,_7,_8,_9,_10',
 'NU_ANO,NO_MUNICIPIO_RESIDENCIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,NU_NOTA_COMP5,NU_NOTA_COMP4,NU_NOTA_COMP3,NU_NOTA_COMP2',
 '2016,Jo�o Pessoa,M,0,3,1,120,120,100,120',
 '2016,Vitorino,M,0,1,1,80,120,120,120',
 '2016,Salvador,M,0,3,1,100,80,80,100']

### Map para obter somente as colunas de Sexo e numero da nota 2:

In [24]:
rddEnemMapGroup = rdd.map(lambda x: (x.split(",")[2],x.split(",")[9]))

In [25]:
rddEnemMapGroup.take(5)

[('_3', '_10'),
 ('TP_SEXO', 'NU_NOTA_COMP2'),
 ('M', '120'),
 ('M', '120'),
 ('M', '100')]

### Função para validar dados da coluna de nota 2 que não contem registro:

In [26]:
def notContainFloat(line):
    try:
        tmp = (line[0],float(line[1]))
        return tmp
    except:
        return ("",0)

In [27]:
rddEnemMapSex = rddEnemMapGroup.map(lambda line: notContainFloat(line))

#### Map dos dados para 1 = Masculino, 0 = Feminino:

In [28]:
rddEnemMapSex = rddEnemMapSex.map(lambda line: (1,line[1]) if line[0] == "M" 
                                          else (0,line[1]) )

In [29]:
numPartitions = rddEnemMapSex.getNumPartitions()
print("Total partitions: {}".format(numPartitions))

Total partitions: 55


#### Função para transformar linhas em hash:

In [30]:
def partitioner(line):
    return hash(line)

### Particionado em 2 distribuições pela chave:

In [31]:
rddEnemMapSexBy = rddEnemMapSex.partitionBy(2, partitioner)

In [32]:
numPartitions = rddEnemMapSexBy.getNumPartitions()
print("Total partitions: {}".format(numPartitions))

Total partitions: 2


### Total somatório da nota 2 pela chave de sexo:

In [33]:
%%time
dfagregateBykey = rddEnemMapSexBy.aggregateByKey(0,
                                       lambda x, y: x + y,
                                       lambda x, y: x + y)
dfagregateBykey.take(5)

Wall time: 1min 7s


[(0, 385835196.0), (1, 269400931.0)]

In [36]:

schema = StructType([
    StructField("NU_ANO",IntegerType(), True),
    StructField("NO_MUNICIPIO_RESIDENCIA",StringType(), True),
    StructField("TP_SEXO",StringType(), True),
    StructField("TP_ESTADO_CIVIL",IntegerType(), True),
    StructField("TP_COR_RACA",IntegerType(), True),
    StructField("TP_NACIONALIDADE",IntegerType(), True),
    StructField("NU_NOTA_COMP5",IntegerType(), True),
    StructField("NU_NOTA_COMP4",IntegerType(), True),
    StructField("NU_NOTA_COMP3",IntegerType(), True),
    StructField("NU_NOTA_COMP2",IntegerType(), True)]
)

In [38]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
df = spark.read.csv("./Datasets/enem_importcoluns_load", header=True, sep=",", schema=schema)

In [39]:
from pyspark.sql.types import DoubleType

In [40]:
df.show()

+------+-----------------------+-------+---------------+-----------+----------------+-------------+-------------+-------------+-------------+
|NU_ANO|NO_MUNICIPIO_RESIDENCIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|NU_NOTA_COMP5|NU_NOTA_COMP4|NU_NOTA_COMP3|NU_NOTA_COMP2|
+------+-----------------------+-------+---------------+-----------+----------------+-------------+-------------+-------------+-------------+
|  2016|                Itabuna|      F|              0|          1|               1|            0|          120|          120|          120|
|  2016|                    Exu|      F|              0|          3|               1|         null|         null|         null|         null|
|  2016|            Sete Lagoas|      M|              0|          3|               1|           20|          120|          100|          120|
|  2016|                 Trairi|      M|              0|          3|               1|         null|         null|         null|         null|
|  201

In [41]:
df2 = df.select('NU_ANO', 'NO_MUNICIPIO_RESIDENCIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 
                'TP_NACIONALIDADE', 'NU_NOTA_COMP5', 'NU_NOTA_COMP4', 'NU_NOTA_COMP3', 'NU_NOTA_COMP2')

In [42]:
df2 = df2.withColumn("NU_NOTA_COMP2", df2["NU_NOTA_COMP2"].cast(DoubleType()))

In [43]:
df2.createOrReplaceTempView("tmp")

In [44]:
df3 = df2.select("TP_SEXO","NU_NOTA_COMP2").groupBy("TP_SEXO").sum("NU_NOTA_COMP2")

In [45]:
%%time
for x in df3.take(4): print(x)

Row(TP_SEXO='TP_SEXO', sum(NU_NOTA_COMP2)=None)
Row(TP_SEXO='F', sum(NU_NOTA_COMP2)=385835196.0)
Row(TP_SEXO='M', sum(NU_NOTA_COMP2)=269400931.0)
Wall time: 6.72 s


### Média da nota 2 por ano e município:

In [46]:
%%time
spark.sql("""SELECT NU_ANO, NO_MUNICIPIO_RESIDENCIA, avg(NU_NOTA_COMP2) AS NOTA2, COUNT(*) AS Registros 
          FROM tmp
          group by nu_ano, no_municipio_residencia
          order by avg(nu_nota_comp2) desc
          """).show()

+------+-----------------------+------------------+---------+
|NU_ANO|NO_MUNICIPIO_RESIDENCIA|             NOTA2|Registros|
+------+-----------------------+------------------+---------+
|  2016|           Carlos Gomes|131.57894736842104|       28|
|  2016|                Niter�i| 130.0946179306375|    19869|
|  2016|   S�o Jo�o das Duas...|             130.0|       50|
|  2016|            Vila Flores|129.84126984126985|       78|
|  2016|      Aparecida d'Oeste|129.04109589041096|      108|
|  2016|   Santa Cec�lia do Sul|128.64864864864865|       51|
|  2016|   Vista Alegre do P...|128.57142857142858|       34|
|  2016|        Dores de Campos| 128.1045751633987|      439|
|  2016|        Estrela d'Oeste|             127.5|      169|
|  2016|    S�o Domingos do Sul|             127.5|       80|
|  2016|          Nova Friburgo|126.85108695652174|     5162|
|  2016|                   Ermo|125.94594594594595|       56|
|  2016|               Ariranha|          125.8125|      125|
|  2016|

In [48]:
df_avg_notas = spark.sql("""SELECT NU_ANO, NO_MUNICIPIO_RESIDENCIA, avg(NU_NOTA_COMP2) AS NOTA2, COUNT(*) AS Registros 
          FROM tmp
          group by nu_ano, no_municipio_residencia
          order by avg(nu_nota_comp2) desc
          """)

#### Carregando dados de populacional por municipio:

In [50]:
df_populacao_mun = spark.read.csv("./Datasets/estimativa_TCU_2016_20170614.csv", header=True, sep=";")

In [51]:
df_populacao_mun.show(5)

+---+-------+----------+--------------------+------------------+----+----+
| UF|COD. UF|COD. MUNIC|   NOME DO MUNIC�PIO|POPULA��O ESTIMADA| _c5| _c6|
+---+-------+----------+--------------------+------------------+----+----+
| RO|     11|     00015|Alta Floresta D'O...|            25.506|null|null|
| RO|     11|     00023|           Ariquemes|           105.896|null|null|
| RO|     11|     00031|              Cabixi|             6.289|null|null|
| RO|     11|     00049|              Cacoal|            87.877|null|null|
| RO|     11|     00056|          Cerejeiras|            17.959|null|null|
+---+-------+----------+--------------------+------------------+----+----+
only showing top 5 rows



In [53]:
oldColumns = df_populacao_mun.schema.names[:-2]
newColumns = ["UF","COD_UF","COD_MUNIC","NOME_MUNIC","POPULACAO"]

In [54]:
for x in zip(oldColumns,newColumns): df_populacao_mun = df_populacao_mun.withColumnRenamed(x[0],x[1])

In [55]:
df_populacao_mun.show(5)

+---+------+---------+--------------------+---------+----+----+
| UF|COD_UF|COD_MUNIC|          NOME_MUNIC|POPULACAO| _c5| _c6|
+---+------+---------+--------------------+---------+----+----+
| RO|    11|    00015|Alta Floresta D'O...|   25.506|null|null|
| RO|    11|    00023|           Ariquemes|  105.896|null|null|
| RO|    11|    00031|              Cabixi|    6.289|null|null|
| RO|    11|    00049|              Cacoal|   87.877|null|null|
| RO|    11|    00056|          Cerejeiras|   17.959|null|null|
+---+------+---------+--------------------+---------+----+----+
only showing top 5 rows



In [56]:
df_populacao_mun.createOrReplaceTempView("ibge")

In [59]:
%pip install jupyter-dash
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### Analise de média das notas 2 por municipio relacionando com dataset de quantidade populacional por municipio:

In [60]:
dfAnalise = spark.sql("""SELECT A.NU_ANO, A.NO_MUNICIPIO_RESIDENCIA, avg(A.NU_NOTA_COMP2) AS MEDIA_NOTA2, COUNT(*) AS Registros,
          B.POPULACAO, (COUNT(*) / B.POPULACAO) PERCENT_POPUL_REGI
          FROM tmp AS A
          LEFT JOIN ibge AS B ON (B.NOME_MUNIC = A.NO_MUNICIPIO_RESIDENCIA)
          group by A.nu_ano, A.no_municipio_residencia, B.POPULACAO
          order by avg(A.nu_nota_comp2) desc
          """).toPandas()

In [61]:
dfAnalise.head()

,NU_ANO,NO_MUNICIPIO_RESIDENCIA,MEDIA_NOTA2,Registros,POPULACAO,PERCENT_POPUL_REGI
0,2016.0,Carlos Gomes,131.578947,28,1.560,17.948718
1,2016.0,Niter�i,130.094618,19869,497.883,39.906966
2,2016.0,S�o Jo�o das Duas Pontes,130.000000,50,2.607,19.179133
3,2016.0,Vila Flores,129.841270,78,3.373,23.124815
4,2016.0,Aparecida d'Oeste,129.041096,108,4.362,24.759285


In [7]:
import sys, os, json 
from pyspark.streaming import StreamingContext

In [8]:
NotasSchema = StructType().add("NO_MUNICIPIO_RESIDENCIA","string").add("NOTA2","double")

### Inicializando streamFile para leitura de novas notas:

In [9]:
novasNotasDf = spark.readStream.option("sep", ";").option("header", "true").schema(NotasSchema).csv("./Datasets/new_notas")

In [10]:
novasNotasDf.createOrReplaceTempView("novasnotas")

In [11]:
totalMunicipio = spark.sql("SELECT NO_MUNICIPIO_RESIDENCIA, SUM(NOTA2) AS TOTAL_NOTA_2 FROM novasnotas GROUP BY NO_MUNICIPIO_RESIDENCIA")

In [12]:
totalMunicipio = totalMunicipio.coalesce(4)

In [13]:
## ou = novasNotasDf.groupBy("NU_MUNICIPIO_RESIDENCIA").sum("TOTAL_NOTA_2")

In [14]:
query = totalMunicipio.writeStream.outputMode("complete").format("console").start()

In [ ]:
query.awaitTermination()